In [1]:
import argparse
import time

import cv2

from processing import extract_parts, draw

from config_reader import config_reader
from model.cmu_model import get_testing_model
import easydict
import glob
import numpy as np

Using TensorFlow backend.


In [3]:
def crop(image, w, f):
    return image[:, int(w * f): int(w * (1 - f))]

In [ ]:
tic = time.time()
file_list = glob.glob('input_data/*.jpg') 
number = 0

for i in range(len(file_list)):    
    
    factor = 0.1
    scale_search = [0.22, 0.25, .5, 1, 1.5, 2]  
    process_speed = 1
    scale_search = scale_search[0:process_speed]
    params['scale_search'] = scale_search
    resize_fac = 0.6

    model = get_testing_model()
    model.load_weights('model/keras/model.h5')
    params, model_params = config_reader()
    input_image = cv2.imread(file_list[i])

    black = cv2.imread('black.png')
    
    width = input_image.shape[1] 

    frame = crop(input_image, width, factor)
    input_image = cv2.resize(frame, (0, 0), fx=1/resize_fac, fy=1/resize_fac, interpolation=cv2.INTER_CUBIC)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)

    body_parts, all_peaks, subset, candidate = extract_parts(input_image, params, model, model_params)
    black = draw(black, all_peaks, subset, candidate)
    
    toc = time.time()
    cv2.imwrite('output_data/test%d.jpg'%number, black)
    print('processing time is %.5f' % (toc - tic))
    number += 1

cv2.destroyAllWindows()

processing time is 11.45638
processing time is 22.53911
